In [1]:
using LowLevelFEM, LinearAlgebra

In [2]:
gmsh.initialize()
#gmsh.open("torus.geo")

In [3]:
R = 10

setParameter("R", R)

gmsh.merge("sphere-part.geo")

Info    : Reading 'sphere-part.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 20%] Meshing curve 2 (Circle)
Info    : [ 40%] Meshing curve 3 (Circle)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Line)
Info    : [ 90%] Meshing curve 6 (Line)
Info    : Done meshing 1D (Wall 0.00119808s, CPU 0.001116s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (BSpline surface, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0808362s, CPU 0.078095s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 487 nodes...
Info    : Done tetrahedrizing 495 nodes (Wall 0.00884729s, CPU 0.006717s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary

In [4]:
mat = material("volu")
prob = Problem([mat])

Problem("sphere-part", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("volu", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0e-7, 0.1, 1.0)], 1.0, 4296, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [5]:
vx(x, y, z) = x * y * z / 100
vy(x, y, z) = (x^2 + y^2 + z^2) / 10
vz(x, y, z) = (2 * x * y + 2 * y * z + 2 * z * 1) / 10

vz (generic function with 1 method)

In [6]:
t = tangentVector(prob, "peri")
n = normalVector(prob, "surf")
v_peri = VectorField(prob, "peri", [vx, vy, vz])
v_surf = VectorField(prob, "surf", [vx, vy, vz]);
v_volu = VectorField(prob, "volu", [vx, vy, vz]);

In [7]:
integrate(prob, "peri", v_peri ⋅ t)

66.66670977825976

In [8]:
integrate(prob, "surf", (∇ × v_surf) ⋅ n)

0.0

In [9]:
rot_v = ∇ × v_volu
rot_v = nodesToElements(elementsToNodes(rot_v), onPhysicalGroup="surf")
integrate(prob, "surf", rot_v ⋅ n)

66.65341526288174

In [10]:
integrate(prob, "peri", (x, y, z) -> 1)

47.12388828415764

In [11]:
R * π / 2 * 3

47.12388980384689

In [12]:
integrate(prob, "surf", (x, y, z) -> 1)

150.20718999221245

In [13]:
R^2 * π * 4 / 8

157.07963267948966

In [14]:
showElementResults(t, name="t")
showElementResults(n, name="n")
showElementResults(v_peri, name="v peri")
showElementResults(v_surf, name="v surf")
showElementResults(v_peri ⋅ t, name="v_t")
showElementResults(rot(v_surf), name="rot v")
#showDoFResults(elementsToNodes(rot(v_surf)), name="rot v")

6

In [15]:
r = VectorField(prob, "surf", [(x, y, z) -> x, (x, y, z) -> y, (x, y, z) -> z])
ω = VectorField(prob, "surf", [1, 2, 3])
v = ω × r
ω2 = ∇ × v / 2

showDoFResults(ω, name="ω")
showDoFResults(v, name="v")
showDoFResults(ω2, name="ω2")
ω = VectorField(prob, "volu", [1, 2, 3])
showDoFResults(ω2 ⋅ ω, name="ωn")

10

In [16]:
r = VectorField(prob, "volu", [(x, y, z) -> x, (x, y, z) -> y, (x, y, z) -> z])
ω = VectorField(prob, "volu", [1, 2, 3])
v = ω × r
ω2 = ∇ × v / 2

showDoFResults(ω, name="ω")
showDoFResults(v, name="v")
showDoFResults(ω2, name="ω2")

13

In [17]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
Fontconfig warning: using without calling FcInit()


In [18]:
gmsh.finalize()